## Installing Important Packages

In [1]:
!pip install datasets
!pip install transformers
!pip install evaluate
!pip install jiwer
!pip install kenlm
!pip install pyctcdecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 63.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 34.0 MB/s eta 0:00

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


# Section 1: Automatic Speech Recognition

## 1.1 Import important libraries

In [4]:
from datasets import Dataset, Audio
from transformers import pipeline
import pandas as pd
import evaluate
import torch
import time

## 1.2 Load Urdu Recipes dataset

In [56]:
df = pd.read_csv('urdu-recipes.csv')
df.head()

,Id,recipe_name,additional_phrase,ing_name,Ing_quantity,Ing_unit,source,audio
0,1,بلوچی گوشت,بلوچی گوشت بنانے کے اجزاء,NaN,NaN,NaN,kfoods,dataset\Balochi Gosht - kfoods.wav
1,1,بلوچی گوشت,NaN,بون لیس مٹن,½,کلو,kfoods,dataset\Balochi Gosht - kfoods.wav
2,1,بلوچی گوشت,NaN,ادرک لہسن کا پیسٹ,1,کھانے کا چمچہ,kfoods,dataset\Balochi Gosht - kfoods.wav
3,1,بلوچی گوشت,NaN,ثابت لال مرچ,6,عدد,kfoods,dataset\Balochi Gosht - kfoods.wav
4,1,بلوچی گوشت,NaN,ہری مرچ,3,عدد,kfoods,dataset\Balochi Gosht - kfoods.wav


## 1.3 Preprocessing dataset

In [57]:
df['Ing_quantity'] = df['Ing_quantity'].replace(r' ', '', regex= True)
df['ing_name'] = df['ing_name'].str.strip('\t')
df = df.fillna('').astype(str)
df = df.replace(r'[\n\xa0\u200c]', '', regex = True)

### Converting numeric quantities into words

In [58]:
df.Ing_quantity.unique()

array(['', '½', '1', '6', '3', '1/4', '2', '300', '50', '5', '4', '200',
       '1-1½', '1½', '150', '8', '12', '½-2', '100', '10', '½-1', '15',
       '6-8', '3-4', '250', '125', '14-16', '10-15', '8-10', '2-3', '¼',
       '20-25', '7-8', '1/2'], dtype=object)

In [59]:
urdu_counting = {'1/4': 'ایک چوتھائی', '1/2': 'آدھا', '½-1': 'ڈیڑھ','1½': 'ڈیڑھ', '½-2': 'ڈھائی',
 '2½': 'ڈھائی', '½': 'آدھا', '1': 'ایک', '2': 'دو', '3': 'تین', '4': 'چار', '5': 'پانچ', '6': 'چھ',
 '7': 'سات', '8': 'آٹھ', '9': 'نو', '10': 'دس', '11': 'گیارہ', '12': 'بارہ', '13': 'تیرہ', '14': 'چودہ',
 '15': 'پندرہ', '16': 'سولہ', '17': 'سترہ', '18': 'اٹھارہ', '19': 'انیس', '20': 'بیس', '21': 'اکیس', '22': 'بائیس',
 '23': 'تئیس', '24': 'چوبیس', '25': 'پچیس', '26': 'چھببیس', '27': 'ستائیس', '28': 'اٹھائیس', '29': 'انتیس', '30': 'تیس',
 '31': 'اکتیس', '32': 'بتیس', '33': 'تینتیس', '34': 'چونتیس', '35': 'پینتیس', '36': 'چھتیس', '37': 'سینتیس', '38': 'اڑتیس',
 '39': 'انتالیس', '40': 'چالیس', '41': 'اکتالیس', '42': 'بیالیس', '43': 'تینتالیس', '44': 'چوالیس', '45': 'پینتالیس', '46': 'چھیالیس',
 '47': 'سینتالیس', '48': 'اڑتالیس', '49': 'انچاس', '50': 'پچاس', '51': 'اکیاون', '52': 'باون', '53': 'ترپن', '54': 'چون', 
 '55': 'پچپن', '56': 'چھپن', '57': 'ستاون', '58': 'اٹھاون', '59': 'انسٹھ', '60': 'ساٹھ', '61': 'اکسٹھ', '62': 'باسٹھ',
 '63': 'ترسٹھ', '64': 'چوسٹھ', '65': 'پینسٹھ', '66': 'چھیاسٹھ', '67': 'سڑسٹھ', '68': 'اٹھسٹھ', '69': 'انہتر', '70': 'ستر',
 '71': 'اکہتر', '72': 'بہتر', '73': 'تہتر', '74': 'چوہتر', '75': 'پچہتر', '76': 'چھہتر', '77': 'ستتر', '78': 'اٹھہتر',
 '79': 'اناسی', '80': 'اسی', '81': 'اکیاسی', '82': 'بیاسی', '83': 'تراسی', '84': 'چوراسی', '85': 'پچاسی', '86': 'چھیاسی',
 '87': 'ستاسی', '88': 'اٹھاسی', '89': 'نواسی', '90': 'نوے', '91': 'اکانوے', '92': 'بانوے', '93': 'ترانوے', '94': 'چورانوے',
 '95': 'پچانوے', '96': 'چھیانوے', '97': 'ستانوے', '98': 'اٹھانوے', '99': 'ننانوے', '100': 'سو', '125':'ایک سو پچیس' , '150': 'ایک سو پچاس', '200': 'دو سو',
 '250': 'دو سو پچاس', '300': 'تین سو', '350': 'تین سو پچاس', '400': 'چار سو',
 '450': 'چار سو پچاس', '500': 'پانچ سو'}

In [60]:
def num_to_words(value): 
    if '-' in value:
        tmp = value.split('-')
        if tmp[0] != '½':
            tmp[0] = urdu_counting[tmp[0]]
            tmp[1] = urdu_counting[tmp[1]]
            return (' سے '.join(tmp))           
    
    if value in urdu_counting.keys():
        return urdu_counting[value]
    return  value
for i in df.index:
    df.loc[i, 'Ing_quantity'] = num_to_words(df.loc[i, 'Ing_quantity'])
    df.loc[i, 'ing_name'] = df.loc[i, 'ing_name'].replace('14', num_to_words('14'))
    df.loc[i, 'ing_name'] = df.loc[i, 'ing_name'].replace('16', num_to_words('16'))

In [61]:
df.Ing_quantity.unique()

array(['', 'آدھا', 'ایک', 'چھ', 'تین', 'ایک چوتھائی', 'دو', 'تین سو',
       'پچاس', 'پانچ', 'چار', 'دو سو', 'ایک سے ڈیڑھ', 'ڈیڑھ',
       'ایک سو پچاس', 'آٹھ', 'بارہ', 'ڈھائی', 'سو', 'دس', 'پندرہ',
       'چھ سے آٹھ', 'تین سے چار', 'دو سو پچاس', 'ایک سو پچیس',
       'چودہ سے سولہ', 'دس سے پندرہ', 'آٹھ سے دس', 'دو سے تین', '¼',
       'بیس سے پچیس', 'سات سے آٹھ'], dtype=object)

### Combine necessary columns to create one complete Ingredient Phrase for each recipe

In [62]:
df['text'] = df[['additional_phrase', 'ing_name', 'Ing_quantity', 'Ing_unit']].astype(str).apply(' '.join, axis=1)
chars_to_remove_regex = r'[\ ِ\'ُ\،\?\.\!\-\;\:\"\“\%\‘\”\�\'\)\(]| +\n'
df['text'] = df['text'].replace(chars_to_remove_regex, ' ', regex = True).replace(r' +', ' ', regex = True)
df['audio'] = df["audio"].replace(r"\\", "/", regex = True)
df.head()

,Id,recipe_name,additional_phrase,ing_name,Ing_quantity,Ing_unit,source,audio,text
0,1,بلوچی گوشت,بلوچی گوشت بنانے کے اجزاء,,,,kfoods,dataset/Balochi Gosht - kfoods.wav,بلوچی گوشت بنانے کے اجزاء
1,1,بلوچی گوشت,,بون لیس مٹن,آدھا,کلو,kfoods,dataset/Balochi Gosht - kfoods.wav,بون لیس مٹن آدھا کلو
2,1,بلوچی گوشت,,ادرک لہسن کا پیسٹ,ایک,کھانے کا چمچہ,kfoods,dataset/Balochi Gosht - kfoods.wav,ادرک لہسن کا پیسٹ ایک کھانے کا چمچہ
3,1,بلوچی گوشت,,ثابت لال مرچ,چھ,عدد,kfoods,dataset/Balochi Gosht - kfoods.wav,ثابت لال مرچ چھ عدد
4,1,بلوچی گوشت,,ہری مرچ,تین,عدد,kfoods,dataset/Balochi Gosht - kfoods.wav,ہری مرچ تین عدد


In [63]:
new_df = df.groupby(['Id', 'recipe_name', 'audio'])['text'].apply(' '.join).reset_index()

In [64]:
new_df.head()

,Id,recipe_name,audio,text
0,1,بلوچی گوشت,dataset/Balochi Gosht - kfoods.wav,بلوچی گوشت بنانے کے اجزاء بون لیس مٹن آدھا ک...
1,10,فروٹی اسموتھی,dataset/Fruity Smoothie - kfoods.wav,فروٹی اسموتھی بنانے کے اجزاء آڑو ایک کپ در...
2,100,چلی چلی جھینگے,dataset/Chilli Chilli Jheengay - kfoods.wav,چلی چلی جھینگے بنانے کے اجزاء جھینگے آدھا کلو ...
3,11,کرسپ اسپنچ وِد سیلیڈ,dataset/Crisp Chicken with Salad - kfoods.wav,کرسپ اسپنچ و د سیلیڈ بنانے کے اجزاء چکن بریس...
4,12,کریم آلو,dataset/Cream Aloo - kfoods.wav,کریم آلو بنانے کے اجزاء آلو ایک کلو ہری پیا...


## 1.4 Convert the audio recordings into float arrays for model input using 🤗 Datasets

In [65]:
dataset = Dataset.from_pandas(new_df)
dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
sampling_rate = dataset.features["audio"].sampling_rate

In [66]:
dataset[0]

{'Id': '1',
 'recipe_name': 'بلوچی گوشت',
 'audio': {'path': 'dataset/Balochi Gosht - kfoods.wav',
  'array': array([-1.55635487e-11, -6.55808090e-12, -9.75002371e-12, ...,
          2.75824248e-04, -2.55801511e-04, -4.05652361e-04]),
  'sampling_rate': 16000},
 'text': 'بلوچی گوشت بنانے کے اجزاء   بون لیس مٹن آدھا کلو  ادرک لہسن کا پیسٹ ایک کھانے کا چمچہ  ثابت لال مرچ چھ عدد  ہری مرچ تین عدد  پسا ہوا زیرہ ایک چائے کا چمچہ  ہر ا دھنیا ایک چوتھائی گٹھی  دہی ایک پاﺅ  نمک حسب ذائقہ  پساگرم مصالحہ ایک چائے کا چمچہ  پسا کھوپرا دو کھانے کا چمچہ  تیل تین کھانے کا چمچہ  پیاز ایک عدد  جائفل جاوتری پاﺅڈر آدھا چائے کا چمچہ  شملہ مرچ ایک عدد  کڑی پتے چند عدد  پودین ایک چوتھائی گٹھی  پسی لال مرچ ایک چائے کا چمچہ  ہلدی ایک چائے کا چمچہ  ا بلے کٹے ٹماٹر دو عدد'}

## 1.5 Generate audio transcriptions using 🤗 pre-trained Automatic Speech Recognition models and 🤗 Pipelines

In [68]:
audios = []
for i in range (dataset.num_rows):
  audios.append(dataset[i]['audio']['array'])

### Using Whisper fine-tuned model on urdu from publicly available models 

In [78]:
asr_whisper = pipeline("automatic-speech-recognition", model="ihanif/whisper-medium-urdu", chunk_length_s=30, stride_length_s=(4, 2), device = "cuda:0", batch_size=10)

In [ ]:
start = time.time()

whisper = asr_whisper(audios)

end = time.time()

In [80]:
print(f'Time taken for whisper model: {end - start} seconds')

Time taken for whisper model: 2448.114921808243 seconds


In [94]:
whisper = [item['text'] for item in whisper]

In [95]:
whisper[0]

'بلوچی گوشڈر آدہ چائے کا چمچہ شملا مرچ ایک ادد کڑی پتے چند ادد پودینا ایک چوتائی کٹی پسی لال مرچ ایک چائے کا چمچہ ہلدی ایک چائے کا چمچہ ابلی کٹے ٹیمارٹر دو ادد'

### Using Wav2Vec2 fine-tuned model on urdu from publicly available models 

In [71]:
asr_wav2vec2 = pipeline("automatic-speech-recognition", model="anuragshas/wav2vec2-xls-r-300m-ur-cv9-with-lm", chunk_length_s=30, stride_length_s=(4, 2), device = "cuda:0", batch_size=2)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [72]:
start = time.time()

wav2vec2 = asr_wav2vec2(audios)

end = time.time()

In [73]:
print(f'Time taken for wav2vec2 model: {end - start} seconds')

Time taken for wav2vec2 model: 119.16069173812866 seconds


In [ ]:
wav2vec2 = [item['text'] for item in wav2vec2]

In [93]:
wav2vec2[0]

'بلوچی گوشت بنانے کے سا مول لیس مٹ دھکیل ادرک لہسن کا پیسٹ ایک کھانے کا م ثابت لال میچ چھت ہری مر تین اد صا ہوا زیر ایک جائے مچا ہرا دنیا ایک جاتا ئی کردی دحی ایک با نمک ہسپانی کا پیسہ گرم مثالی ایک جائے جا پیسہ کو پرا دو کھانے کے جمے تیل تین کھانے کے جمے پیاز ایک عدد جائیفلجوتری پاؤڈر ادا جائے گا چمجاشملہ میٹ ایک عدد کڑی پتی چند دت پہ دینا ایک اتاری کی سی لال میٹ ایک جائے کا چمچہ الدی ایک چھائے کا چمچہ بلی کٹی ٹماٹر دو عدد'

### Display generated Audio Transcriptions

In [96]:
recipe_names = new_df['recipe_name'].values.tolist()
gold_references = new_df['text'].values.tolist()

In [97]:
predictions_df = pd.DataFrame({'Recipe Name': recipe_names, 'Original Text': gold_references, 'Whisper Predicted Text': whisper, 'Wav2Vec2 Predicted Text': wav2vec2})

In [98]:
predictions_df['Whisper Predicted Text'] = predictions_df["Whisper Predicted Text"].replace(r'،', '', regex = True)
predictions_df['Wav2Vec2 Predicted Text'] = predictions_df["Wav2Vec2 Predicted Text"].replace(r'،', '', regex = True)

In [99]:
predictions_df.head()

,Recipe Name,Original Text,Whisper Predicted Text,Wav2Vec2 Predicted Text
0,بلوچی گوشت,بلوچی گوشت بنانے کے اجزاء بون لیس مٹن آدھا ک...,بلوچی گوشڈر آدہ چائے کا چمچہ شملا مرچ ایک ادد...,بلوچی گوشت بنانے کے سا مول لیس مٹ دھکیل ادرک ل...
1,فروٹی اسموتھی,فروٹی اسموتھی بنانے کے اجزاء آڑو ایک کپ در...,فروٹی سمودی بنانے کے اتزا اڑو ایک کپ ترمیانے گ...,فروٹی اس مودی بنانے کے عجز اڑو ایک کپ درمیانے ...
2,چلی چلی جھینگے,چلی چلی جھینگے بنانے کے اجزاء جھینگے آدھا کلو ...,چلی چلی جھنگے بنانے کے اجزا جھنگے آدھا کلو نار...,چلی چلی جین گے بنانے کے جزا جیگ دھکیل ناریل قو...
3,کرسپ اسپنچ وِد سیلیڈ,کرسپ اسپنچ و د سیلیڈ بنانے کے اجزاء چکن بریس...,کرسپ سپنچ ویٹ سالٹ بنانے کے اجزاء چکن بریسٹ دو...,کرسپسپنچوت سالٹ بنانے کے از چکن برس دو عدد کھی...
4,کریم آلو,کریم آلو بنانے کے اجزاء آلو ایک کلو ہری پیا...,کریم آلو بنانے کے عدزہ آلو آدھا کلو ہری پیاز د...,کریم آلو بنانے کے عصا آلو ادا کیلوں ہری پیاز د...


In [100]:
predictions_df.to_csv('Speech2TextPredictions.csv')

## 1.6 Evaluating Word Error Rate (WER) for wav2vec2 and whisper predictions

In [101]:
cer = evaluate.load('cer')
wer = evaluate.load('wer')

In [102]:
whisper_preds = predictions_df['Whisper Predicted Text'].values.tolist()
wav2vec2_preds = predictions_df['Wav2Vec2 Predicted Text'].values.tolist()

In [103]:
wer_whisper = 100* wer.compute(predictions=whisper_preds, references = gold_references)
cer_whisper = 100* cer.compute(predictions=whisper_preds, references = gold_references)
wer_whisper, cer_whisper

(56.08238257519632, 39.50892857142857)

In [104]:
wer_wav2vec2 = 100* wer.compute(predictions=wav2vec2_preds, references = gold_references)
cer_wav2vec2  = 100* cer.compute(predictions=wav2vec2_preds, references = gold_references)
wer_wav2vec2 , cer_wav2vec2 

(60.64602163283449, 25.800945378151262)